- import

In [1]:
import matplotlib.pyplot as plt
import os, tempfile, librosa, pyaudio
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.models import load_model
from PIL import Image
import tensorflow as tf
import pathlib
from scipy import signal
import keras.utils as image
from pydub import AudioSegment
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.applications import MobileNetV2, VGG19
import sounddevice as sd
import time
import pandas as pd

2023-10-12 15:54:00.756557: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


- time taken to load model

In [3]:
start = time.time()
model = tf.keras.models.load_model('../models/ann_mobilenetv2_1_sec.h5')
base_model = MobileNetV2(include_top=False, weights="imagenet", input_shape=(224, 224, 3))
if not os.path.exists(os.path.join('temp')):
    os.mkdir(os.path.join('temp'))
end = time.time()
print('load the model',f'>>>>{round((end-start),4)}')

2023-10-12 15:58:17.834302: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-12 15:58:18.304581: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-12 15:58:18.305617: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

load the model >>>>6.437


- Load the ground truth from csv

In [ ]:
fd = pd.read_csv('actual_label.csv')
asd =fd['actual_label']
asd =asd.to_list()

- This function calculates and returns time for operations like:

- convert audio to spectrogram 
- Create, Render Canvas and convert to NumPy array
- Resize the array
- extract features from the image using mobilenetv2
- Prediction by the model 
- total time taken for execution

In [4]:
def process_audio(aud_data, sr,count):
    print(f'>>>>>>>>>>>>>>>>> for the {count} chunk >>>>>>>>>>>>>>>>>')

#     play the audio
#     sd.play(aud_data, sr)
#     sd.wait()
#     time.sleep(1.5)

    strt = time.time()
    
    # Convert sample into spectogram
    start = time.time()
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
    ms = librosa.feature.melspectrogram(y=aud_data, sr=8000)
    log_ms = librosa.power_to_db(ms, ref=np.max)
    librosa.display.specshow(log_ms, sr=RATE)
    end = time.time()
    first = end-start
#     Create, Render Canvas and convert to NumPy array
    start = time.time()
    canvas = fig.canvas
    canvas.draw()
    width, height = canvas.get_width_height()
    image_array = np.frombuffer(canvas.tostring_rgb(), dtype='uint8')
    image_array = image_array.reshape(height, width, 3)
    end = time.time()
    second = end-start

#   Resize the array
    start = time.time()
    image = tf.image.resize(image_array, (224,224)) 
    x = np.expand_dims(image, axis=0)
    x = preprocess_input(np.array(x))
    y = base_model.predict(x, verbose=False)
    end = time.time()
    third = end-start
#   model prediction 
    start = time.time()
    predictions = model.predict(y, verbose=False)
    res = np.argmax(predictions)
    print(class_labels[res])
    plt.axis('off')
    end = time.time()
    fourth =end-start
    
    endd = time.time()
    last = endd-strt
    plt.close('all')
    return class_labels[res],first,second,third,fourth,last

- load the audio file
- take 2 second chunk and get the time intervals for operations
- append the results in a list
- repeat step 2 and 3 till the end of audio

In [ ]:
RATE = 8000
lst =[]
count =1
c=0
RATE = 8000
class_labels = ['IVR', 'Music', 'Speech']
audio_file=r'test_audio_1sec.wav'
audio_data, sr = librosa.load(audio_file,sr =8000)


while True: 
    r = 8000 * 1
    if len(audio_data)>r:
        audio_data1 = audio_data[:r]
        audio_data =audio_data[r:]
        res,first,second,third,fourth,last =process_audio(audio_data1, sr,count)
        lst.append([asd[c],res,first,second,third,fourth,last])
        count+=1
        c+=1
    else:
        res,first,second,third,fourth,last =process_audio(audio_data, sr,count)
        lst.append([asd[c],res,first,second,third,fourth,last])
        break

- convert the list to dataframe with column names

In [27]:
data =pd.DataFrame(lst,columns =['actual','predicted','audio to spectogram','convert to array','Resize the array','model prediction','total_time'])

In [ ]:
data

- export the dataframe results to CSV

In [26]:
data.to_csv('results/preprocessing_time_reduce_io_1sec.csv')